In [2]:
import requests;import math;import multiprocessing
import random;from joblib import Parallel, delayed;from collections import ChainMap;from matplotlib.patches import Patch
num_cores = 5;import pandas as pd;import numpy as np
import seaborn as sb;import matplotlib.pyplot as plt;import matplotlib as mpl;import glob;import os
from matplotlib_venn import venn2;import gzip;import shutil;from tqdm import tqdm_notebook as tq;import warnings
import scipy.stats as stats;import pdb;
warnings.simplefilter('ignore');cmapSpecial = mpl.colors.LinearSegmentedColormap.from_list("", ["blue",'white',"red"])
cmapSpecial2 = mpl.colors.LinearSegmentedColormap.from_list("", ["red",'white',"blue"])
cmapSpecial3 = mpl.colors.LinearSegmentedColormap.from_list("", ['white',"red"])
cmapSpecial4 = mpl.colors.LinearSegmentedColormap.from_list("", [(0,'black'),(0.4999,'grey'),(0.5,"blue"),
                                                                 (0.75,'white'),(1,"red")],N=256)
__debug=False;mpl.rcParams['figure.dpi'] = 300;plt.rcParams["figure.facecolor"] = "w";import time

The main 2 OmniPath functions included here are:  
runOmniPath and  
OmniPathPaths  

runOmniPath is the original basic version:    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;It takes a list of sourceGenes and a list of targetGenes.   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Alterantively you can give it a dataframe with source and target columns, in that case sourceGenes should be the name of the source column, targetGenes should be the name of the target column, and the dataFrame parameter should be set to True (its default is False).  
  
OmniPathPaths is the more complex, multi path version:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Like runOmniPath it also takes the sourceGenes and targetGenes lists   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Alongside a maxSteps number, which repersents maximum number of connections that will be searched for.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;For example maxSetps of 1 is a direct connection, while maxSteps of 2 will allow for 1 intermidiate gene between the source and target (if a path with less connections that maxSteps exists it will still be found).  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;As well as a maxPaths number, which will limit how many parallel paths will be allowed to be found.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;For example if there are 5 different paths from geneA to geneB but maxPaths is set to 4, one of those paths will not be included in the output (based on which paths were found first by the program).  
  
Both runOmniPath and OmniPathPaths will both search omnipath.org for any paths between all the source and target genes   &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;runOmniPath will return a single dataframe with all the direct connections between those genes  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;OmniPathPaths will return a list of dataframes of each source to target gene connections (as it assumes the multipaths will lead to a lot of paths between per each pair) along with the name of the connection the dataframe shows.  
  
Both functions can also save the dataframes produced.    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;To do so the save parameter should be set to True.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;and saveDirectory should be set to the directory you want the excel files to be saved in.

In [99]:
# Uncomment to see the inputs for each function:
# help(OmniPathPaths)
# help(runOmniPath)

In [98]:
def OmniPathPaths(genesFrom,genesTo,saveDirectory='',save=False,maxSteps=1,maxPaths=100):
    directory=saveDirectory
    maxSteps -= 1
    sourceGenes=genesFrom;targetGenes=genesTo;dataFrame=False
    sourceGenes,targetGenes = fixSourceTargetTFs(sourceGenes,targetGenes,dataframe=dataFrame,string=False)
    dfLst=[]
    for sourceG in tq(sourceGenes,desc='Searching OmniPaths'):
        inputs=targetGenes
        out = Parallel(n_jobs=10)(delayed(findPathInOmni)(sourceG,targetG,maxSteps=maxSteps,
                                                          pathsWanted=maxPaths) for targetG in inputs)
        for tup in out:
            if tup:
                val,path,g = tup
                name=directory+'/'+sourceG+' to '+g+'.xlsx';name=name.replace(' -> ',' ')
                if val and not os.path.exists(name):
                    dfAll=recurGetSer(path)
                    if len(dfAll.index)>0:
                        dfAll,totalPaths,pathLens=cleanupOmniDf(dfAll,g)
                        if save:
                            dfAll.to_excel(name)
                        name2=name.replace('.xlsx','')
                        name2=name2.split('/')[-1]
                        dfLst.append((name2,dfAll))
    return dfLst

def cleanupOmniDf(dfAllIn,targetGene):
    dfAll=dfAllIn.copy()
    foundLst = list(dfAll[dfAll['target_genesymbol']==targetGene].index)
    nName=0
    groupLst=[]
    pathLen=[]
    frst=True
    for n in range(len(foundLst)):
        if frst:
            elem = foundLst[n]
            pathLen.append(elem)
            for i in range(elem+1):
                groupLst.append(n)
            frst=False
        else:
            elem=foundLst[n]
            elemPrev=foundLst[n-1]
            pathLen.append(elem-elemPrev)
            for i in range(elemPrev+1,elem+1):
                groupLst.append(n)
        nName+=1
    dfAll.insert(0,'Path Group',groupLst)
    totalPaths = nName
    return dfAll,totalPaths,pathLen

def recurGetSer(path,done=True):
    outPut=[]
    for elem in path:
        if type(elem) is list:
            out2 = recurGetSer(elem,False)
            outPut.extend(out2)
        else:
            if type(elem) is pd.core.series.Series:
                elem=pd.DataFrame(elem).T
            if "b'source" in elem.columns and "target" in elem.columns:
                elem=elem.drop(columns=["b'source","target"])
            outPut.append(elem)
    if done:
        df = pd.concat(outPut,ignore_index=True)
        return df
    else:
        return outPut
    
def findPathInOmni(sourceG,targetG,maxSteps,pathsWanted):
    return findPathInOmniHelper(sourceG,targetG,maxSteps,step=0,lst=[],doneLst=[],found=0,pathsWanted=pathsWanted)
def findPathInOmniHelper(sourceG,targetG,maxSteps=4,step=0,lst=[],doneLst=[],found=0,pathsWanted=1):
    if step > maxSteps:
        return False,[],targetG
    else:
        try:
            OmniPathURL='https://omnipathdb.org/interactions/?genesymbols=1&sources='+sourceG
            OmniPathURL+='&fields=sources,references'
            __r = requests.get(OmniPathURL);page = str(__r.content);df=__OmniPathHelperFunc(page)
            
            step+=1
            doneLst.append(sourceG)
            if type(df)==pd.core.frame.DataFrame:
                if targetG in list(df['target_genesymbol']):
                    connect = df[df['target_genesymbol']==targetG]
                    lst.append(connect)
                    return True,lst,targetG
                else:
                    outLst=[]
                    if step <= maxSteps:
                        for row in df.index:
                            lst2=lst.copy();ser=df.loc[row];lst2.append(ser)
                            gene = df.loc[row]['target_genesymbol']
                            if not gene in doneLst:
                                val, out,g = findPathInOmniHelper(gene,targetG,maxSteps=maxSteps,step=step,lst=lst2,
                                                                  doneLst=doneLst,found=found,pathsWanted=pathsWanted)
                                if val:
                                    found+=1
                                    outLst.append(out)
                                    if found>pathsWanted:
                                        break
                    if len(outLst)>0:
                        return True,outLst,targetG
                    else:
                        return False,outLst,targetG
            else:
                return False,[],targetG
        except:
            findPathInOmniHelper(sourceG,targetG,maxSteps,step,lst,doneLst,found,pathsWanted)

def findOmnipathPathsFaster(sourceGene,targetGene,directory,maxSteps):
    name=directory+'/'+sourceGene+' '+targetGene+'.xlsx';name=name.replace(' -> ',' ')
    if os.path.exists(name):
        out=(False,[],sourceGene)
    else:
        doneLst=[targetGene];gLst=[sourceGene]
        rowsDict={};connectionsDict={};foundTarget=False
        for num in range(maxSteps):
            gLst = [i for i in gLst if not i in doneLst]
            df = fasterOmniConnection(gLst)
            if type(df)==pd.core.frame.DataFrame:
                for row in df.index:
                    ser=df.loc[row]
                    surce = ser['source_genesymbol']
                    targ = ser['target_genesymbol']
                    if not surce in doneLst and surce != targetGene:
                        rowsDict[(surce,targ)]=ser
                        if surce in connectionsDict:
                            connectionsDict[surce].append(targ)
                        else:
                            connectionsDict[surce]=[targ]
                doneLst.extend(gLst)
                if targetGene in list(df['target_genesymbol']):
                    foundTarget=True
                gLst=list(df['target_genesymbol'])
            else:break
        if foundTarget:
            out=recurFindConnections(sourceGene,targetGene,connectionsDict,rowsDict,maxSteps)
        else:out=(False,[],sourceGene)
    return out
def cleanOmniConnectionLst(lst):
    while True:
        out=[];allGood=True;n=-1
        for i in range(len(lst)-1):
            n=i;_,g1 = lst[i];g2,__= lst[i+1]
            if g1 != g2:allGood=False
            else:out.append(lst[i])
        out.append(lst[n+1])
        if allGood:break
        lst = out
    return out
def getBottomLst(lst):
    if type(lst[0])==list and len(lst)==1:
        return getBottomLst(lst[0])
    else:
        return lst
def recurFindConnections(startG,findG,connectionsDict,rowsDict,maxNum):
    return recurFindConnectionsHelper(startG,findG,connectionsDict,rowsDict,lst=[],doneLst=[findG],maxNum=maxNum)
def recurFindConnectionsHelper(gene,findG,connectionsDict,rowsDict,lst,doneLst,num=0,maxNum=4):
    if num > maxNum or not gene in connectionsDict:
        return False,[], findG
    else:
        doneLst.append(gene)
        if findG in connectionsDict[gene]:
            lst.append((gene,findG))
            lst = cleanOmniConnectionLst(lst)
            rowsLst=[]
            for g1,g2 in lst:
                rowsLst.append(rowsDict[(g1, g2)])
            return True, rowsLst,findG
        else:
            num+=1
            outLst=[]
            for i in connectionsDict[gene]:
                if not i in doneLst:
                    lst.append((gene,i))
                    val, out, g = recurFindConnectionsHelper(gene=i,findG=findG,connectionsDict=connectionsDict,
                                                             rowsDict=rowsDict,lst=lst,doneLst=doneLst,num=num)
                    if val:
                        outLst.append(out)
                    else:
                        lst=lst[:-1]
                else:
                    val=False
            if len(outLst)>0:
                return True,outLst,findG
            else:
                return False,[],findG
            
def runOmniPath(sourceGenes,targetGenes,dataFrame=False,save=False,saveDirectory=''):
    sourceTFs,targetTFs = fixSourceTargetTFs(sourceGenes,targetGenes,dataframe=dataFrame)
    dfOut = runOmniPathHelper(sourceTFs,targetTFs)
    if type(dfOut) is pd.core.frame.DataFrame:
        dfOut = dfOut.drop(columns=["b'source",'target','dip_url'])
    if save:
        name=saveDirectory+'/'+sourceTFs+' to '+targetTFs+'.xlsx';name=name.replace(' -> ',' ')
        dfOut.to_excel(name)
    return dfOut
def fasterOmniConnection(geneLst):
    splt=int(round(len(geneLst)/800,0))
    if splt:
        dfLst=[]
        for i in range(splt):
            gLst = geneLst[i*800:i+1*800]
            gS = ','.join(gLst)
            df=fasterOmniConnectionHelper(gS)
            if type(df)==pd.core.frame.DataFrame:
                dfLst.append(df)
        if len(dfLst)>0:
            dfAll = pd.concat(dfLst)
            dfAll.index=range(len(dfAll.index))
        else:
            dfAll = False
        return dfAll
    else:
        gS = ','.join(geneLst)
        df=fasterOmniConnectionHelper(gS)
        if type(df)==pd.core.frame.DataFrame:
            dfAll=df
        else:
            dfAll=False
    return dfAll
def fasterOmniConnectionHelper(gS):
    try:
        OmniPathURL='https://omnipathdb.org/interactions/?genesymbols=1&sources='+gS
        OmniPathURL+='&fields=sources,references'
        __r = requests.get(OmniPathURL);page = str(__r.content);df=__OmniPathHelperFunc(page)
        return df
    except:
        return fasterOmniConnectionHelper(gS)
    
def runOmniPathHelper(geneFrom,geneTo):
    try:
        OmniPathURL='https://omnipathdb.org/interactions/?genesymbols=1&source_target=AND&sources='+geneFrom
        OmniPathURL+='&targets='+geneTo+'&fields=sources,references'
        __r = requests.get(OmniPathURL);page = str(__r.content);df=__OmniPathHelperFunc(page)
    except:
        return runOmniPathHelper(geneFrom,geneTo)
    return df
def __OmniPathHelperFunc(page):
    totalLst=[];n=0
    for line in page.split('\\n'):
        cols=line.split('\\t');totalLst.append(cols)
    if len(totalLst)>2:
        for ls in totalLst:
            if len(ls)>1:n+=1
        totalLst=totalLst[:n]
        try:
            df=pd.DataFrame(totalLst);df.columns=df.loc[0];df=df.T;df.drop(columns=[0],inplace=True);df=df.T
            return df
        except:
            return False
    else: 
        return False
def fixSourceTargetTFs(source,target,tfs=True,dataframe=True,string=True):
    if dataframe:
        if tfs:
            source=source[source['type']=='tf'];target=target[target['type']=='tf']
        sourceTFs=set(source['name']);targetTFs=set(target['name']);common=sourceTFs.intersection(targetTFs)
        sourceTFs=list(sourceTFs);sourceTFs = [i.split('_')[0] for i in sourceTFs if not i in targetTFs]
        targetTFs =list(targetTFs);targetTFs = [i.split('_')[0] for i in targetTFs if not i in sourceTFs]
    else:
        sourceTFs=source.copy();targetTFs=target.copy()
    sourceTFs = list(set(sourceTFs));targetTFs = list(set(targetTFs))
    if string:
        sourceTFs=','.join(sourceTFs);targetTFs=','.join(targetTFs)
    return sourceTFs,targetTFs